In [3]:
import pandas as pd

file_path = "~/messages_backup_24_2025.csv.gz" # ../../messages_copy_12_22_25.csv.zst" #"../../messages.csv.gz"
#chunk_size = 500_000  # Adjust based on available RAM


In [4]:
import json

In [5]:

data_iterator = pd.read_csv(file_path, chunksize=chunk_size)

for idx, chunk in enumerate(data_iterator):
    # Process your chunk (e.g., filtering, simple math)
    #processed_chunk = chunk[chunk['value'] > 50]
    #print(len(chunk))
    #print(idx)
    #if idx > 200:
    #    break
    # Save to a new file (append mode)
    #processed_chunk.to_csv('output.csv', mode='a', index=False, header=not os.path.exists('output.csv'))

    #print(chunk)
    
    if idx > 1:
          break

In [10]:
%pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import dask.dataframe as dd
from tqdm import tqdm

In [1]:
output_dir = "messages_parquet_zstd"
os.makedirs(output_dir, exist_ok=True)

NameError: name 'os' is not defined

In [12]:
# simple version just directly copies to parquet!!!

import pandas as pd
import os

output_dir = "messages_parquet_zstd"
os.makedirs(output_dir, exist_ok=True)

# Process in 1-2GB chunks (uncompressed size)
# For 3TB, aim for ~1000-2000 small parquet files
chunk_size = 1_000_000 

for i, chunk in tqdm(enumerate(pd.read_csv(file_path, chunksize=chunk_size))):
    # Convert to efficient dtypes (e.g., float32, category) to save space
    # chunk['col'] = chunk['col'].astype('category') 
    
    file_path = f"{output_dir}/part_{i:04d}.parquet"
    chunk.to_parquet(file_path, engine='pyarrow', compression='zstd')
    print(f"Saved chunk {i}")

FileNotFoundError: [Errno 2] No such file or directory: 'massive_data.csv.gz'

In [3]:
# fancy version converts from json/python data types to ones that can be loaded more efficiently!!!
import os
import pandas as pd
import json
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

file_path = "/home/pwhiddy/messages_backup_24_2025.csv.gz"
output_dir = "/home/pwhiddy/messages_parquet_zstd"
os.makedirs(output_dir, exist_ok=True)

# 1. Initialize the CSV reader (iterator)
reader = pd.read_csv(file_path, chunksize=400_000) # Smaller chunks for JSON overhead
error_log = "processing_errors.log"

def process_chunk_optimized(chunk):
    def parse(msg):
        try:
            d = json.loads(msg)
            m = d.get('metadata', {})
            
            # Extract and sanitize
            user = str(m.get('user', '')).strip()
            color = str(m.get('color', '')).strip()
            extra = str(m.get('extra', '')).strip()
            
            # Convert empty strings to None for Arrow/Parquet safety
            return (
                user if user != '' else None,
                color if color != '' else None,
                extra if extra != '' else None,
                d.get('coords', [])
            )
        except Exception as e:
            with open(error_log, "a") as f:
                f.write(f"Error in chunk {i}: {str(e)}\n")
            return (None, None, None, [])

    parsed = [parse(m) for m in chunk['message']]
    users, colors, extras, coords = zip(*parsed)

    # BUILD THE DATAFRAME
    new_df = pd.DataFrame({
        # .reset_index(drop=True) is the key fix here!
        'timestamp': pd.to_datetime(chunk['timestamp'], format='ISO8601').reset_index(drop=True),
        'user': pd.Series(users, dtype='category'),
        'color': pd.Series(colors, dtype='category'),
        'extra': pd.Series(extras),
        'coords': coords 
    })
    return new_df

for i, chunk in tqdm(enumerate(reader)):

    try:
        final_chunk = process_chunk_optimized(chunk)
        
        # 3. Write with PyArrow optimizations
        final_chunk.to_parquet(
            f'{output_dir}/part_{i:06d}.parquet', 
            engine='pyarrow', 
            compression='zstd',
            compression_level=17, 
            # Utilize all cores for column-wise compression
            #use_threads=True,
            index=False
        )

    except Exception as e:
        with open(error_log, "a") as f:
            f.write(f"Error in chunk {i}: {str(e)}\n")
        continue # Move to the next chunk instead of crashing

1854it [42:48:58, 83.14s/it] 


error: Error -3 while decompressing data: invalid code lengths set